In [ ]:
#from google.colab import drive
#drive.mount('/content/drive')

import  pandas as pd
from numpy import set_printoptions
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import f_classif
from sklearn.feature_selection import VarianceThreshold
from sklearn.model_selection import train_test_split
import numpy as np
# load data

#import os
#for dirname, _, filenames in os.walk('../input'):
#    for filename in filenames:
#        print(os.path.join(dirname, filename))
pd.set_option('max_columns', None) 
df_train=pd.read_csv("./drive/MyDrive/TrainingWiDS2021.csv")
df_test=pd.read_csv("./drive/MyDrive/UnlabeledWiDS2021.csv")

cats = []
for c in list(df_train.columns):
  unique = df_train[c].nunique()
  if unique <= 5:
      # decide this is a category
      newcol, code = pd.factorize(df_train[c], na_sentinel=None)
      cats.append(code)
      df_train[c] = newcol
cols = df_train.select_dtypes([np.number]).columns          ## string features with more than 5 categories are thrown out
X = df_train[cols] 

X.fillna(X.mean(), inplace=True)
"""dropped_na = []                                        #only numeric data is kept - skipping admission ward data - some binary columns 
for c in list(X.columns):
  if X[c].isna().mean()>.5:
    X.drop(c, axis = 1, inplace=True)
    dropped_na.append(c)"""


                                      #replace nan with mean of column - binary columns now trinary
X.drop(['encounter_id', 'hospital_id', 'icu_id'], axis = 1, inplace = True) #id has no affect on diabetes
                                 
def correlation(dataset, threshold):                        #function calculates the corplot, gathers columns that are correlated into corr_features
    col_corr = set()  # Set of all the names of correlated columns
    corr_matrix = dataset.corr()
    for i in range(len(corr_matrix.columns)):
        for j in range(i):
            if abs(corr_matrix.iloc[i, j]) > threshold: 
                colname = corr_matrix.columns[i]        
                col_corr.add(colname)
    return col_corr
corr_features = correlation(X, 0.7)
X.drop(corr_features,axis=1, inplace=True)                   #remove correlated columns


FileNotFoundError: ignored

# New Section
reads in data from local drive, cleans up a bit

removes correlated data
replaces nan with mean of column
string factors with more than 5 levels are thrown out

next section keeps only columns determined to be useful according to Karen's analysis

In [ ]:
X.columns

Index(['Unnamed: 0', 'age', 'bmi', 'elective_surgery', 'gender', 'height',
       'icu_admit_source', 'icu_stay_type', 'pre_icu_los_days',
       'readmission_status', 'albumin_apache', 'apache_2_diagnosis',
       'arf_apache', 'bilirubin_apache', 'bun_apache', 'creatinine_apache',
       'fio2_apache', 'gcs_eyes_apache', 'gcs_motor_apache',
       'gcs_unable_apache', 'gcs_verbal_apache', 'glucose_apache',
       'heart_rate_apache', 'hematocrit_apache', 'intubated_apache',
       'map_apache', 'paco2_apache', 'pao2_apache', 'ph_apache',
       'resprate_apache', 'sodium_apache', 'temp_apache', 'urineoutput_apache',
       'ventilated_apache', 'wbc_apache', 'd1_diasbp_invasive_max',
       'd1_diasbp_invasive_min', 'd1_diasbp_max', 'd1_diasbp_min',
       'd1_heartrate_min', 'd1_mbp_invasive_max', 'd1_resprate_max',
       'd1_resprate_min', 'd1_spo2_max', 'd1_spo2_min',
       'd1_sysbp_invasive_max', 'd1_sysbp_invasive_min', 'd1_temp_max',
       'h1_diasbp_invasive_max', 'h1_diasb

In [ ]:
X_f=X.drop(labels=['diabetes_mellitus'],axis=1)         #X full
y_f=X['diabetes_mellitus']                              #y full

X_train, X_validation, Y_train, Y_validation = train_test_split(X_f, y_f, test_size=0.30, random_state=1)

from sklearn.preprocessing import StandardScaler

sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_validation = sc.transform(X_validation)

In [ ]:
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
import matplotlib.pyplot as plt

# Make predictions on validation dataset
model = LinearDiscriminantAnalysis()
model.fit(X_train, Y_train)
predictions = model.predict(X_validation)
print(accuracy_score(Y_validation, predictions))
print(confusion_matrix(Y_validation, predictions))
print(classification_report(Y_validation, predictions))

0.806340913747183
[[ 2298  6134]
 [ 1428 29188]]
              precision    recall  f1-score   support

           0       0.62      0.27      0.38      8432
           1       0.83      0.95      0.89     30616

    accuracy                           0.81     39048
   macro avg       0.72      0.61      0.63     39048
weighted avg       0.78      0.81      0.78     39048



In [ ]:
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
model = QuadraticDiscriminantAnalysis()
model.fit(X_train, Y_train)
predictions = model.predict(X_validation)
print(accuracy_score(Y_validation, predictions))
print(confusion_matrix(Y_validation, predictions))
print(classification_report(Y_validation, predictions))
# QDA underperforms LDA because of the high # of predictor# keep only gaussian distr. data

/usr/local/lib/python3.6/dist-packages/sklearn/discriminant_analysis.py:691: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")


0.744698832206515
[[ 3655  4777]
 [ 5192 25424]]
              precision    recall  f1-score   support

           0       0.41      0.43      0.42      8432
           1       0.84      0.83      0.84     30616

    accuracy                           0.74     39048
   macro avg       0.63      0.63      0.63     39048
weighted avg       0.75      0.74      0.75     39048



In [ ]:
from sklearn.linear_model import LogisticRegression
model = LogisticRegression()
model.fit(X_train, Y_train)
predictions = model.predict(X_validation)
print(accuracy_score(Y_validation, predictions))
print(confusion_matrix(Y_validation, predictions))
print(classification_report(Y_validation, predictions))

0.8055726285597213
[[ 2087  6345]
 [ 1247 29369]]
              precision    recall  f1-score   support

           0       0.63      0.25      0.35      8432
           1       0.82      0.96      0.89     30616

    accuracy                           0.81     39048
   macro avg       0.72      0.60      0.62     39048
weighted avg       0.78      0.81      0.77     39048



In [ ]:
from sklearn import svm
model = svm.SVC()
model.fit(X_train, Y_train)
predictions = model.predict(X_validation)
print(accuracy_score(Y_validation, predictions))
print(confusion_matrix(Y_validation, predictions))
print(classification_report(Y_validation, predictions))

0.8147152222905142
[[ 2167  6265]
 [  970 29646]]
              precision    recall  f1-score   support

           0       0.69      0.26      0.37      8432
           1       0.83      0.97      0.89     30616

    accuracy                           0.81     39048
   macro avg       0.76      0.61      0.63     39048
weighted avg       0.80      0.81      0.78     39048



By consensus of the group, LDA was chosen as the model of choice. Although Logistic Regression and SVC also did well, SVC took more time and Logistic Regression theory is dependent on Gaussian distributed data (which we do not have)